## Summarizing top posts specific to an APT#

### 1. extract data for each APT

In [38]:
import pandas as pd
from os import listdir

path = '../data/'
files = listdir('../data/')
df = pd.DataFrame(columns=["url", "query", "text"])

aptNumbers = [1, 3, 5, 10, 12, 15, 16, 17, 18, 19, 27, 28, 29, 30, 32, 33, 34, 35, 37, 38]

for n in aptNumbers:
    for f in files:
        temp = pd.read_csv(path + f)
        df = df.append(temp)
    aptNum = "APT%s" % n
    df.loc[df['query'] == aptNum]
    apt_json = aptNum+'.json'
    df.to_json(apt_json, orient='records')

### 2. summarizing the posts

In [39]:
import json
import nltk
import numpy

BLOG_DATA = "APT3.json"

N = 100  # Number of words to consider
CLUSTER_THRESHOLD = 5  # Distance between words to consider
TOP_SENTENCES = 5  # Number of sentences to return for a "top n" summary

# Approach taken from "The Automatic Creation of Literature Abstracts" by H.P. Luhn

def _score_sentences(sentences, important_words):
    scores = []
    sentence_idx = -1

    for s in [nltk.tokenize.word_tokenize(s) for s in sentences]:

        sentence_idx += 1
        word_idx = []

        # For each word in the word list...
        for w in important_words:
            try:
                # Compute an index for where any important words occur in the sentence.

                word_idx.append(s.index(w))
            except ValueError, e: # w not in this particular sentence
                pass

        word_idx.sort()

        # It is possible that some sentences may not contain any important words at all.
        if len(word_idx)== 0: continue

        # Using the word index, compute clusters by using a max distance threshold
        # for any two consecutive words.

        clusters = []
        cluster = [word_idx[0]]
        i = 1
        while i < len(word_idx):
            if word_idx[i] - word_idx[i - 1] < CLUSTER_THRESHOLD:
                cluster.append(word_idx[i])
            else:
                clusters.append(cluster[:])
                cluster = [word_idx[i]]
            i += 1
        clusters.append(cluster)

        # Score each cluster. The max score for any given cluster is the score 
        # for the sentence.

        max_cluster_score = 0
        for c in clusters:
            significant_words_in_cluster = len(c)
            total_words_in_cluster = c[-1] - c[0] + 1
            score = 1.0 * significant_words_in_cluster \
                * significant_words_in_cluster / total_words_in_cluster

            if score > max_cluster_score:
                max_cluster_score = score

        scores.append((sentence_idx, score))

    return scores

def summarize(txt):
    sentences = [s for s in nltk.tokenize.sent_tokenize(txt)]
    normalized_sentences = [s.lower() for s in sentences]

    words = [w.lower() for sentence in normalized_sentences for w in
             nltk.tokenize.word_tokenize(sentence)]

    fdist = nltk.FreqDist(words)

    top_n_words = [w[0] for w in fdist.items() 
            if w[0] not in nltk.corpus.stopwords.words('english')][:N]

    scored_sentences = _score_sentences(normalized_sentences, top_n_words)

    # Summarization Approach 1:
    # Filter out nonsignificant sentences by using the average score plus a
    # fraction of the std dev as a filter

    avg = numpy.mean([s[1] for s in scored_sentences])
    std = numpy.std([s[1] for s in scored_sentences])
    mean_scored = [(sent_idx, score) for (sent_idx, score) in scored_sentences
                   if score > avg + 0.5 * std]

    # Summarization Approach 2:
    # Another approach would be to return only the top N ranked sentences

    top_n_scored = sorted(scored_sentences, key=lambda s: s[1])[-TOP_SENTENCES:]
    top_n_scored = sorted(top_n_scored, key=lambda s: s[0])

    # Decorate the post object with summaries

    return dict(top_n_summary=[sentences[idx] for (idx, score) in top_n_scored],
                mean_scored_summary=[sentences[idx] for (idx, score) in mean_scored])

blog_data = json.loads(open(BLOG_DATA).read())

post_count = 0
for post in blog_data:
       
    post.update(summarize(post['text']))

    post_count+=1
    
    print 'Post',post_count,'.......................................................................'
    print post['url']
    print
    print 'Top N Summary'
    print '-------------'
    print ' '.join(post['top_n_summary'])
    print
    print 'Mean Scored Summary'
    print '-------------------'
    print ' '.join(post['mean_scored_summary'])
    print

Post 1 .......................................................................
https://www.crowdstrike.com/blog/two-birds-one-stone-panda/

Top N Summary
-------------
ZHANG Shilong (张世龙)
ZHANG was originally introduced by IntrusionTruth as a reciprocal follower of fisherxp’s Twitter account via his own @baobeilong account. NSACE appears to be a national education body that teaches network information security, including offensive activity7. From their latest post, which contains GAO’s Uber receipts, it is clear the group’s information likely goes beyond merely available OSINT data. In addition, FalconIntelligence notes that following the late 2015 Sino-U.S. brief cyber detente, much of the responsibility for western cyber intrusion operations was handed to the MSS as the PLA underwent an extensive reform that is still currently underway, and which is consolidating its military cyber forces under the Strategic Support Force. Many of these adversaries have begun targeting supply chain a

Post 7 .......................................................................
https://www.securityweek.com/poison-ivy-rat-campaign-leverages-new-delivery-techniques

Top N Summary
-------------
Still, the company did say that “espionage is a reasonable assumption for their motives.”
What the newly observed campaign did show, however, was that the actor behind it is up-to-date with recent social engineering and evasion techniques and isn’t shy when it comes to using them. The malware was distributed via Word documents with malicious macros, and the threat actor was using social engineering to trick users into enabling these macros. The malicious documents were delivered via email, claiming to contain instructions for logging into webmail or information regarding a state law proposal, FireEye explains. The shellcode has a custom XOR-based decryption loop that uses a single byte key (0xD4), and was designed to inject the Poison Ivy backdoor into userinit.exe. Previous Columns by Ionut Ar

Post 12 .......................................................................
https://www.securityweek.com/report-highlights-business-risks-drawn-geopolitical-flashpoints

Top N Summary
-------------
In early March, a DHS report described activity under the 'Pleasantly Surprised' campaign spear-phishing commercial entities in the financial, retail and technology sectors. Internally it continues to increase control over cyber activities with new regulations on data flows and VPNs. However, for the moment, it believes that the "re-election of Iranian President Hassan Rouhani is likely to have a stabilizing effect on Iranian cyber activities." In April, Google and Facebook became victims in a scam that netted $100 million for the scammers. Hacktivists
Flashpoint notes a decline in western hacktivism.

Mean Scored Summary
-------------------
China
Chinese state-sponsored activity has remained low following the Xi-Obama agreement made in September 2015. In early March, a DHS report descri

Post 18 .......................................................................
https://www.securityweek.com/china-linked-spies-used-new-malware-uk-government-attack

Top N Summary
-------------
The attack has been attributed to a threat actor known as APT15, Ke3chang, Mirage, Vixen Panda and Playful Dragon. The group has improved its tools and techniques over the years, and NCC recently spotted two new backdoors it created. Experts determined that the hackers had stolen a VPN certificate from a compromised host and used it to regain access via the corporate VPN. In addition to BS2005, RoyalCLI and RoyalDNS, APT15 used custom-built keyloggers, and Microsoft SharePoint and Exchange enumeration and data dumping tools. He worked as a high school IT teacher for two years before starting a career in journalism as Softpedia’s security news reporter.

Mean Scored Summary
-------------------
The attack has been attributed to a threat actor known as APT15, Ke3chang, Mirage, Vixen Panda and Play

Post 24 .......................................................................
https://securelist.com/ksb-threat-predictions-for-2018/83169/

Top N Summary
-------------
This allows us to understand the actual attack surface and attacker tactics and to further hone our hunting and detection to address new attacks. Our assessment is that the total number of mobile malware existing in the wild is likely higher than currently reported, due to shortcomings in telemetry that makes these more difficult to spot and eradicate. While many have grown desensitized to the weight of these breaches, it’s important to understand that the release of PII at scale endangers a fundamental pillar of e-commerce and the bureaucratic convenience of adopting the Internet for important paperwork. Perhaps thus far resilient alternatives like ApplePay will come into vogue as de facto means of insuring identity and transactions, but in the meantime we may see a slowdown in the critical role of the Internet for m

...

Post 447 .......................................................................
https://www.securityweek.com/researchers-say-code-reuse-links-north-koreas-malware

Top N Summary
-------------
In fact, actors operating from the same country have been often observed sharing malware code and infrastructure, which often makes attribution highly problematic. The use of these malware families has been already attributed to the Lazarus Group, which is tracked by the U.S. government as Hidden Cobra. The security researchers also discovered a connection between the Tapaoux (or DarkHotel) malware family and samples from Operation Troy. The only malware that stands apart are the RATs attributed to Group 123, which are linked to one another. Previous Columns by Ionut Arghire:‘DemonBot' Botnet Targets Hadoop ServersLogical Bug in Microsoft Word's 'Online Video' Allows Code ExecutionSOC-as-a-Service Firm Arctic Wolf Networks Raises $45 MillionMultiple Vulnerabilities Patched in ASRock Drivers'TimpD

Post 451 .......................................................................
https://www.crowdstrike.com/blog/crowdstrike-falcon-defeats-gothic-panda-in-mitre-nation-state-emulation-test/

Top N Summary
-------------
For the roughly three decades that the commercial cybersecurity industry has existed, it has struggled with a fundamental problem –  a lack of realistic attack testing to determine whether the products that companies and consumers buy actually work in stopping real “in-the-wild” threats. Today I’m thrilled to announce that CrowdStrike Falcon, backed by our elite Falcon OverWatch™ team, aced the simulation – detecting and providing visibility into nearly every technique tested by the MITRE team. Our OverWatch team detects, stops and notifies our customers of about 400 serious breach attempts every single week – two per hour, on average. The one red cell below – “Exfiltration Over Command and Control Channel” (in this case, DNS) – was the only one missed by Falcon. The r

Post 454 .......................................................................
https://securelist.com/the-naikon-apt-and-the-msnmm-campaigns/70029/

Top N Summary
-------------
The MsnMM Campaigns [pdf]  For over half a decade, the Naikon APT waged multiple attack campaigns on sensitive targets throughout South-eastern Asia and around the South China Sea. Targets and victims included ASEAN governmental agencies and government departments, investment enterprises, military, law enforcement and border control organizations, embassies, university faculties and others. The Naikon attackers attempted to exfiltrate sensitive geo-political, military, and economic data; to intercept communications and to maintain surveillance on their victims throughout the MsnMM campaigns. Their toolset and techniques changed over time in many minor ways, and appear to be run by Chinese-speaking individuals. The consistent list of military, economic, and political targets gave away the actor’s interests.

Me

Post 459 .......................................................................
https://www.securityweek.com/flash-player-flaw-used-apt3-group-added-magnitude-exploit-kit

Top N Summary
-------------
The existence of the heap buffer overflow vulnerability (CVE-2015-3113) that can be exploited for arbitrary code execution was brought to light by FireEye. The security hole, which according to Trend Micro is similar to a previously discovered bug (CVE-2015-3043), exists due to the way Adobe Flash Player parses Flash Video (FLV) files. In late May, an exploit for a Flash Player bug fixed by Adobe on May 11 was added to the Angler exploit kit. “Some criminals might be behind several exploit kits, just like Paunch was behind Blackhole and Cool, and they may use the same exploit in all their kits.”









           (function() {
           var po = document.createElement("script"); po.type = "text/javascript"; po.async = true;
           po.src = "https://apis.google.com/js/plusone.js";
 

Post 462 .......................................................................
https://www.securityweek.com/state-sponsored-attackers-use-web-analytics-reconnaissance

Top N Summary
-------------
The reconnaissance campaign, which FireEye has been tracking since last year, is similar and possibly related to the activities of the Russia-linked advanced persistent threat (APT) group identified as Waterbug (Symantec) and Turla (Kaspersky Lab). FireEye has determined that the more than 100 compromised websites are likely to be visited by people interested in international travel, diplomacy, international economics, energy production and policy, and government matters. The list of targets includes government, embassy, higher education and research, entertainment and culture, NGO, international law, media, consumer goods and retail, energy, construction and engineering, visa services, and high tech websites in tens of countries across the world. Of particular interest appear to be executiv